In [42]:
import pandas as pd
import numpy as np
import scipy.io
import datetime
import os

In [2]:
mat = scipy.io.loadmat('../../IMDB_WIKI/wiki/wiki.mat')

In [3]:
# python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)

In [4]:
mat['wiki'][0][0][5]

array([[array([[111.29109473, 111.29109473, 252.66993082, 252.66993082]]),
        array([[252.4833023 , 126.68165115, 354.53192596, 228.73027481]]),
        array([[113.52, 169.84, 366.08, 422.4 ]]), ...,
        array([[169.88839786,  74.31669472, 235.2534231 , 139.68171997]]),
        array([[1, 1, 1, 1]], dtype=uint8),
        array([[ 92.72633235,  62.0435549 , 230.12083087, 199.43805342]])]],
      dtype=object)

In [5]:
dob = mat['wiki'][0][0][0]
photo_taken_year = mat['wiki'][0][0][1]
full_path = mat['wiki'][0][0][2]
gender = mat['wiki'][0][0][3]
name = mat['wiki'][0][0][4]
face_location = mat['wiki'][0][0][5]
face_score = mat['wiki'][0][0][6]
second_face_score = mat['wiki'][0][0][7]

In [6]:
dob

array([[723671, 703186, 711677, ..., 720620, 723893, 713846]], dtype=int32)

In [43]:
class ExtractData():
    def __init__(self, mat_file):
        self.mat = mat_file
        self.mat = scipy.io.loadmat(mat_file)
        self.get_data()

    def get_data(self):
        self.dob = self.mat['wiki'][0][0][0]
        self.photo_taken_year = self.mat['wiki'][0][0][1][0]
        self.full_path = self.mat['wiki'][0][0][2]
        self.gender = self.mat['wiki'][0][0][3]
        self.name = self.mat['wiki'][0][0][4]
        self.face_location = self.mat['wiki'][0][0][5]
        self.face_score = self.mat['wiki'][0][0][6]
        self.second_face_score = self.mat['wiki'][0][0][7]

    def exchange_date(self):
        dates = []
        for mat_date in self.dob[0]:
            mat_date = int(mat_date)
            date_conversion = datetime.date.fromordinal(int(mat_date)) + datetime.timedelta(days=mat_date%1) - datetime.timedelta(days = 366)
            dates.append(date_conversion)
        self.date_converted = np.array(dates)
    
    def full_path_fix(self):
        paths = []
        for x in self.full_path[0]:
            paths.append(x[0])
        self.paths = np.array(paths)

    def fix_gender(self):
        genders = []
        for x in self.gender[0]:
            try:
                if int(x) == 1:
                    genders.append('M')
                elif int(x) == 0:
                    genders.append('F')
            except:
                genders.append('N')
        self.gender = np.array(genders)
    
    def fix_name(self):
        name = []
        for x in self.name[0]:
            try:
                name.append(x[0])
            except:
                name.append(None)
        self.name = np.array(name)
    
    def fix_s_score(self):
        second_score = []
        for x in self.second_face_score[0]:
            if np.isnan(x) == True:
                second_score.append(None)
            else:
                second_score.append(x)
        self.second_face_score = np.array(second_score)
    
    def fix_f_score(self):
        face_score = []
        for x in self.face_score[0]:
            if np.isnan(x) == True:
                face_score.append(None)
            else:
                face_score.append(x)
        self.face_score = np.array(face_score)
    
    def extract_face_location(self):
        face_location_1 = []
        face_location_2 = []
        for x in self.face_location[0]:
            if len(x) == 1:
                face_location_1.append(f'{str(x[0][0])}, {str(x[0][1])}, {str(x[0][2])}, {str(x[0][3])}')
            else:
                face_location_1.append(f'{str(x[0][0])}, {str(x[0][1])}, {str(x[0][2])}, {str(x[0][3])}')
                face_location_2.append(f'{str(x[1][0])}, {str(x[1][1])}, {str(x[1][2])}, {str(x[1][3])}')
        self.face_location_1 = np.array(face_location_1)

    def create_dataFrame(self, csv_path, csv_file):
        self.exchange_date()
        self.full_path_fix()
        self.fix_gender()
        self.fix_name()
        self.fix_s_score()
        self.fix_f_score()
        self.extract_face_location()
        final_frame = pd.DataFrame({
            'img_path': self.paths,
            'dob': self.date_converted,
            'photo_taken_year': self.photo_taken_year,
            'gender': self.gender,
            'name': self.name,
            'face_location' : self.face_location_1,
            'face_score' : self.face_score
        })
        if os.path.exists(csv_path) == False:
            os.makedirs(csv_path)
        final_frame.to_csv(f'{csv_path}{csv_file}')

In [50]:
obj = ExtractData('../../IMDB_WIKI/wiki/wiki.mat')

In [45]:
obj.create_dataFrame('./','temp.csv')